In [1]:
import pandas as pd
from gemini import  clean_unstructed_text
import unidecode  


# Limpieza

In [13]:
data = pd.read_csv('data/encuesta_raw.csv')
# remover respuesta de prueba
data = data[~(data['Id'] == 1)]
# remover columnas innecesaias
data.drop(['Email', 'Name'], axis=1, inplace=True)

# renombrar las columnas
data.rename(columns={
    'Id': 'response_id',
    'Start time': 'start_response',
    'Completion time': 'complete_response',
    'Edad': 'age',
    'Sexo': 'sex',
    'Año de estudio en el área de la salud.': 'years_studying',
    'Que carrera esta cursando?': 'degree',
    '¿Cómo calificaría el estado actual de la infraestructura hospitalaria en Managua?': 'infrastructure_score',
    '¿Ha notado mejoras en la infraestructura hospitalaria durante el período 2020–2025?': 'notice_infrastructure_improvement',
    'Mencione almenos 3 cambios en la infraestructura hospitalaria que considere relevante que haya llevado a cabo nuestro gobierno revolucionario.': 'infrastructure_changes_raw',
    '¿Con qué frecuencia ha observado el uso de  sistemas digitales (expedientes electrónicos, telemedicina, imágenes  digitales) durante sus prácticas o rotaciones?': 'frecuency_digital_systems',
    '¿Cree que la incorporación de nuevas tecnologías ha mejorado la atención a los pacientes?': 'technology_improves_attention',
    'Mencione una tecnología hospitalaria que considere más importante en la actualidad.': 'most_important_technology_raw',
    'En su experiencia, ¿la modernización hospitalaria ha mejorado la calidad de la atención?': 'modernization_improves_attention',
    '¿Qué aspecto considera que ha mejorado más? ': 'improved_aspect_raw',
    '¿Qué desafíos persisten en la atención hospitalaria a pesar de la modernización?': 'challenges_raw',
    '¿Cuáles son las patologías más frecuentes que ha observado en sus prácticas?': 'pathologies',
    '¿Considera que la red hospitalaria está mejor preparada para atender estas patologías en comparación con hace 5 años?': 'improved_pathologies_treatments',
    'En general, ¿cómo calificaría la modernización de la red hospitalaria pública en Managua (2020–2025)?  ': 'modernization_score',
    '¿Qué recomendación daría para mejorar la modernización hospitalaria en Nicaragua?': 'recomendations_raw'
}, inplace=True)

data['degree'] = data['degree'].replace(
    'Doctor en Medicina y Cirugía', 'Medicina y Cirugía')


# Normalizar el texto de las variables abiertas
def normalize_text(text: str):
    if pd.isna(text):
        return text
    text = unidecode.unidecode(text.replace('ñ', 'ni'))
    text = text.lower().strip()
    text = ' '.join(text.split())
    return text


data["infrastructure_changes_raw"] = data["infrastructure_changes_raw"].apply(
    normalize_text)
data["most_important_technology_raw"] = data["most_important_technology_raw"].apply(
    normalize_text)
data["improved_aspect_raw"] = data["improved_aspect_raw"].apply(normalize_text)
data["challenges_raw"] = data["challenges_raw"].apply(normalize_text)
data["recomendations_raw"] = data["recomendations_raw"].apply(normalize_text)


# corregir tipos de datos
data = data.astype({
    'response_id': 'string',
    'start_response': 'datetime64[ns]',
    'complete_response': 'datetime64[ns]',
    'age': 'category',
    'sex': 'category',
    'years_studying': 'category',
    'degree': 'category',
    'infrastructure_score': 'category',
    'notice_infrastructure_improvement': 'category',
    'infrastructure_changes_raw': 'string',
    'frecuency_digital_systems': 'category',
    'technology_improves_attention': 'category',
    'most_important_technology_raw': 'string',
    'modernization_improves_attention': 'category',
    'improved_aspect_raw': 'string',
    'challenges_raw': 'string',
    'pathologies': 'category',
    'improved_pathologies_treatments': 'category',
    'modernization_score': 'int64',
    'recomendations_raw': 'string'

})

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, 1 to 109
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   response_id                        109 non-null    string        
 1   start_response                     109 non-null    datetime64[ns]
 2   complete_response                  109 non-null    datetime64[ns]
 3   age                                109 non-null    category      
 4   sex                                109 non-null    category      
 5   years_studying                     109 non-null    category      
 6   degree                             109 non-null    category      
 7   infrastructure_score               109 non-null    category      
 8   notice_infrastructure_improvement  109 non-null    category      
 9   infrastructure_changes_raw         109 non-null    string        
 10  frecuency_digital_systems          109 non-

## Agregar categorias a las carreras

In [14]:
DEGREE_CATEGORY_MAP = {
    # Personal Médico y Quirúrgico
    "Medicina y Cirugía": "Personal Médico y Quirúrgico",
    "Licenciatura en Anestesia y Reanimación": "Personal Médico y Quirúrgico",

    # Personal de Enfermería
    "Licenciatura en Enfermería": "Personal de Enfermería",

    # Personal de Diagnóstico y Laboratorio
    "Licenciatura en Bioanálisis Clínico": "Personal de Diagnóstico y Laboratorio",
    "Licenciatura en Microbiología": "Personal de Diagnóstico y Laboratorio",
    "Técnico Superior en Citología Cervical": "Personal de Diagnóstico y Laboratorio",

    # Personal de Terapia y Rehabilitación
    "Licenciatura en Fisioterapia": "Personal de Terapia y Rehabilitación",
    "Licenciatura en Nutrición": "Personal de Terapia y Rehabilitación",

    # Personal de Atención Especializada
    "Odontología": "Personal de Atención Especializada",
    "Licenciatura en Optometría Médica": "Personal de Atención Especializada",
    "Podología": "Personal de Atención Especializada",

    # Personal de Salud Pública y Prevención
    "Técnico Superior en Higiene y Epidemiología": "Personal de Salud Pública y Prevención"
}


data['degree_category'] = data['degree'].apply(
    lambda x: DEGREE_CATEGORY_MAP[x]).astype('category')

# Limpiar campos de preguntas abiertas

In [16]:
question = "Mencione cambios en la infraestructura hospitalaria que considere relevante"
result_df = clean_unstructed_text(question, data['infrastructure_changes_raw'].to_list())
result_df.columns = ['infrastructure_changes', 'infrastructure_answer_tag']
data[result_df.columns] = result_df.values

data[["infrastructure_changes", "infrastructure_answer_tag"]]

,infrastructure_changes,infrastructure_answer_tag
1,[atencion],Neutral
2,[],Off-topic
3,"[banios, salas de espera, salas de enfermos]",Positive
4,[sala hospitalaria solo para personas oncologi...,Positive
5,[mejor comodidad del paciente],Positive
...,...,...
105,"[ampliacion del area de emergencia en el han, ...",Positive
106,[maquinas de anestesia bien equipadas],Positive
107,[],Off-topic
108,"[nuevos edificios hospitalarios, nuevas salas ...",Positive


In [17]:
question = "Mencione una o mas tecnologías hospitalarias que considere más importante en la actualidad"
result_df = clean_unstructed_text(question, data['most_important_technology_raw'].to_list())
result_df.columns = ['most_important_technology', 'most_important_technology_answer_tag']
data[result_df.columns] = result_df.values

data[["most_important_technology", "most_important_technology_answer_tag"]]

,most_important_technology,most_important_technology_answer_tag
1,[],Off-topic
2,[Aparatos mecanicos para lectura de analisis],Positive
3,[Expedientes digitales],Positive
4,[Expedientes electronicos],Positive
5,"[Disponibilidad de equipos, Diagnosticos digit...",Positive
...,...,...
105,[Clinica movil],Positive
106,[Maquinas de anestesia mas equipadas],Positive
107,[Telemedicina],Positive
108,[Digitalizacion de facturas medicas],Positive


In [18]:
question = "¿Qué aspecto considera que ha mejorado más?"
result_df = clean_unstructed_text(question, data['improved_aspect_raw'].to_list())
result_df.columns = ['improved_aspect', 'improved_aspect_answer_tag']
data[result_df.columns] = result_df.values

data[["improved_aspect", "improved_aspect_answer_tag"]]

,improved_aspect,improved_aspect_answer_tag
1,[acceso a expedientes digitales],Positive
2,"[tiempos de espera, disponibilidad de equipos,...",Positive
3,[acceso a expedientes digitales],Positive
4,[acceso a expedientes digitales],Positive
5,"[disponibilidad de equipos, acceso a expedient...",Positive
...,...,...
105,[disponibilidad de equipos],Positive
106,[disponibilidad de equipos],Positive
107,[disponibilidad de equipos],Positive
108,[disponibilidad de equipos],Positive


In [20]:
question = "¿Qué desafíos persisten en la atención hospitalaria a pesar de la modernización?"
result_df = clean_unstructed_text(question, data['challenges_raw'].to_list())
result_df.columns = ['challenges', 'challenges_answer_tag']
data[result_df.columns] = result_df.values

data[["challenges", "challenges_answer_tag"]]

,challenges,challenges_answer_tag
1,[],Off-topic
2,[trato a los pacientes],Neutral
3,[poca paciencia de los pacientes],Neutral
4,[tiempo de espera del paciente],Neutral
5,[],Off-topic
...,...,...
105,[falta de empatia],Negative
106,[],Off-topic
107,[adopcion de tecnologia],Neutral
108,[caracter del personal],Neutral


In [21]:
question = "¿Qué recomendaciónes daría para mejorar la modernización hospitalaria en Nicaragua?"
result_df = clean_unstructed_text(question, data['recomendations_raw'].fillna('').to_list())
result_df.columns = ['recomendations', 'recomendations_answer_tag']
data[result_df.columns] = result_df.values

data[["recomendations", "recomendations_answer_tag"]]

,recomendations,recomendations_answer_tag
1,[],Off-topic
2,[],Off-topic
3,[],Off-topic
4,[mas tecnologia medica],Positive
5,[mejores equipos],Positive
...,...,...
105,[que se sigan implementando proyectos en mejor...,Positive
106,[],Off-topic
107,[],Off-topic
108,[un personal mas empatico],Positive


# Save data

In [22]:
data.to_parquet('data/encuesta.parquet')